<a href="https://colab.research.google.com/github/bereml/iap/blob/master/libretas/1a_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuronas artificiales

Curso: [Introducción al Aprendizaje Profundo](http://turing.iimas.unam.mx/~ricardoml/course/iap/). Profesores: [Bere](https://turing.iimas.unam.mx/~bereml/) y [Ricardo](https://turing.iimas.unam.mx/~ricardoml/) Montalvo Lezama.

---
---

La neurona artificial es un modelo simplificado de la neurona natural que trata de imitar 3 aspectos principales: 

1. La fuerza sináptica que pondera los impulsos recibidos
2. La acumulación de estos impulsos ponderados 
3. La activación de la neurona que produce un impulso de respuesta a su salida. 

La primera neurona artificial fue la llamada Unidad de Umbral Lineal propuesta en 1943 por Warren McCulloch y Walter Pitts. Este modelo presupone que tanto los valores de los atributos de entrada como los valores de salida son binarios.

In [1]:
# arreglos multidimensionales
import numpy as np

## Unidad de umbral lineal
La operación que lleva a cabo una neurona artificial está dada por la suma pesada evaluada en una función de activación $\phi$.  Una de las primeras funciones de activación utilizadas fue la escalón unitario, definida como

$
\phi(x) = \begin{cases} 1, & \text{si } x > 0\\0, & \text{en caso contrario}\end{cases}
$

Esta se puede implementar en Python de la siguiente manera:

In [2]:
def step(z):
    """Computes step function."""
    return 1.0 if z > 0 else 0.0

Por su parte, la suma pesada simplemente consiste en multiplicar cada entrada por su correspondiente peso y sumarle el sesgo. Esto lo podemos expresar como

$
z = w_1 \cdot x_1 + w_2 \cdot x_2 + \cdots + w_d \cdot x_d + b 
$

En su forma vectorial

$
z = \mathbf{w}^T \mathbf{x} + b
$

Para realizar esto en Python, podemos usar la función [`np.dot`](https://numpy.org/doc/stable/reference/generated/numpy.dot.htmlhttps://www.reddit.com/r/nvidia/comments/lri6as/did_newegg_leak_the_msi_3060_or_was_this_already/https://www.reddit.com/r/nvidia/comments/lri6as/did_newegg_leak_the_msi_3060_or_was_this_already/) de Numpy de la siguiente manera `z = np.dot(w.T, x) + b`. Así, la operación de la neurona completa sería:

In [3]:
def neuron(x, w, b):
    """Returns forward pass as step(`w`·`x`+`b`)."""
    # preactivación
    z = np.dot(w.T, x) + b
    # activación
    a = step(z)
    return a

### Compuerta AND ($\land$)
Esta neurona es capaz de aproximar el operador AND, cuya salida es 1 cuando ambas entradas son 1:

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 0   |
| 0     | 1     | 0   |
| 1     | 0     | 0   |
| 1     | 1     | 1   |

La neurona recibe 2 valores binarios como entrada y produce un valor binario como salida. Específicamente, la neurona calcularía

$\hat{y} = \phi(w_1 \cdot x_1 + w_2 \cdot x_2 + b)$

Para poder aproximar la operación AND es necesario encontrar los valores apropiados de $w_1$, $w_2$ y $b$. Una posible elección sería 10, 10 y -15 respectivamente. Verifiquemos estos valores:

In [4]:
# tabla de verdad
X = np.array([
    [0., 0.], 
    [0., 1.], 
    [1., 0.], 
    [1., 1.]
])

# pesos y sesgo para AND
w = np.array([10, 10]).T
b = -15

# predicción por cada ejemplo
print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    x1, x2 = x = X[i]
    y_hat = neuron(x, w, b)
    print(f'{x1}\t{x2}\t{y_hat}')

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0	0.0	0.0
0.0	1.0	0.0
1.0	0.0	0.0
1.0	1.0	1.0


### Participación: Compuerta OR ($\lor$)

Propon y evalua un conjunto de pesos y sesgo para aproximar la operación OR.

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 0   |
| 0     | 1     | 1   |
| 1     | 0     | 1   |
| 1     | 1     | 1   |

### Participación: Compuerta NAND 
Propon y evalua un conjunto de pesos y sesgo para aproximar la operación NAND.

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 1   |
| 0     | 1     | 1   |
| 1     | 0     | 1   |
| 1     | 1     | 0   |

## Algoritmo del perceptrón

In [5]:
def perceptron_algorithm(X, y, n_epochs=10):
    
    # inicilizamos los pesos y sesgo
    w_new = np.zeros(X.shape[1])
    b_new = 0
    
    # entrenamiento por un numero fijo de épocas
    for i in range(n_epochs):
        
        # error total
        total_error = 0.0
        
        # por cada ejemplo computamos 
        # nuevos pesos y sesgo
        for j in range(X.shape[0]):
            
            # guadamos los anteriores
            w_old = w_new
            b_old = b_new
            
            # computamos la predicción y error
            y_hat = neuron(X[j], w_old, b_old)
            error = y[j] - y_hat
            
            # compuamos nuevos pesos y sesgo
            w_new = w_old + error * X[j]
            b_new = b_old + error

            # agragmos al error total
            total_error += np.abs(error)

        #imprimos el error de la época
        total_error /= float(X.shape[0])
        print(f"Epoch {i}: error = {total_error}")

    # regresamos pesos y sesgo aprendidos
    return w_new, b_new

### Aprendiendo la operación OR

Probemos el algoritmo del perceptrón para aprender la operación lógica OR.

In [6]:
# salida para OR
y_or = np.array([0., 1., 1., 1.]) 

# aprendizaje de pesos y sesgo
w, b = perceptron_algorithm(X, y_or)

# predicción por cada ejemplo
print('\nw_1 = {0}, w_2 = {1}, b = {2}'.format(w[0], w[1], b))
print('-----------------------------')
print('x_1 \tx_2 \t y\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    x1, x2 = x = X[i]
    y_hat = neuron(x, w, b)
    print(f'{x1}\t{x2}\t{y_or[i]}\t{y_hat}')

Epoch 0: error = 0.25
Epoch 1: error = 0.5
Epoch 2: error = 0.25
Epoch 3: error = 0.0
Epoch 4: error = 0.0
Epoch 5: error = 0.0
Epoch 6: error = 0.0
Epoch 7: error = 0.0
Epoch 8: error = 0.0
Epoch 9: error = 0.0

w_1 = 1.0, w_2 = 1.0, b = 0.0
-----------------------------
x_1 	x_2 	 y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	1.0	1.0


### Aprendiendo la operación AND
Ahora veamos qué ocurre si en lugar de la operación OR tratamos de aprender la operación AND

In [7]:
# salida de AND
y_and = np.array([0., 0., 0., 1.])

# aprendizaje de pesos y sesgo
w, b = perceptron_algorithm(X, y_and)

print('\nw_1 = {0}, w_2 = {1}, b = {2}'.format(w[0], w[1], b))
print('-----------------------------')
print('x_1 \tx_2 \t y\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    x1, x2 = x = X[i]
    y_hat = neuron(x, w, b)
    print(f'{x1}\t{x2}\t{y_and[i]}\t{y_hat}')

Epoch 0: error = 0.25
Epoch 1: error = 0.75
Epoch 2: error = 0.75
Epoch 3: error = 0.5
Epoch 4: error = 0.25
Epoch 5: error = 0.0
Epoch 6: error = 0.0
Epoch 7: error = 0.0
Epoch 8: error = 0.0
Epoch 9: error = 0.0

w_1 = 2.0, w_2 = 1.0, b = -2.0
-----------------------------
x_1 	x_2 	 y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	0.0	0.0
1.0	0.0	0.0	0.0
1.0	1.0	1.0	1.0


## Aproximando funciones no lineales: XOR ($\oplus$)
Minsky y Papert mostraron que una neurona del tipo LTU no puede aproximar de forma precisa una función no lineal como la compuerta XOR ($\oplus$):

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 0   |
| 0     | 1     | 1   |
| 1     | 0     | 1   |
| 1     | 1     | 0   |

Sin embargo, es posible aproximar este tipo  combinando múltiples LTU conectadas en red. Por ejemplo, es posible llevar a cabo la operación XOR con operaciones OR, AND y NAND en la siguiente ecuación:

$x_1 \mathbin{\oplus} x_2 = (x_1 \lor x_2) \land \neg(x_1 \land x_2)$

que podemos diagramar de la siguiente forma:


<img src="https://raw.githubusercontent.com/bereml/iap/master/fig/xor.svg" width="350"> 

Esto lo llevamos a cabo con la siguiente función:

In [8]:
# vectorizamos la función para que
# aplique a un arreglo de entradas
step_vec = np.vectorize(step)

# red con dos capas
# capa 1: OR y NAND
# capa 2: AND
def multi_layer(x, W1, b1, W2, b2):
    # capa 1
    z1 = np.dot(W1.T, x) + b1
    a1 = step_vec(z1)
    # capa 2
    z2 = np.dot(W2.T, a1) + b2
    a2 = step_vec(z2)
    return a2

Encontrando los valores de pesos y sesgos adecuados, podemos usar esta función para aproximar la operación XOR. Ya hemos encontrado los pesos y sesgos para las operaciones OR, AND y NAND, por lo que podemos usar estas neuronas con sus correspondientes pesos y sesgos. La red tendría 2 neuronas conectadas a las entradas que realizan las operaciones OR ($w_{11}^{\{1\}} = 10$, $w_{12}^{\{1\}} = 10$ y $b_1^{\{1\}} = -5$)  y NAND ($w_{21}^{\{1\}} = -10$, $w_{22}^{\{1\}} = -10$ y $b_2^{\{1\}} = 15$) respectivamente. La salida de estas 2 neuronas estarían conectadas a una tercera neurona que realiza la operacioón AND ($w_{11}^{\{2\}} = 10$, $w_{12}^{\{2\}} = 10$ y $b_1^{\{2\}} = -15$). En su forma matricial:

$$
\mathbf{W}^{\{1\}} = \left[\begin{matrix} 
10 & -10\\
10 & -10
\end{matrix}\right] 
$$

$$
\mathbf{b}^{\{1\}} = \left[\begin{matrix} 
-5 \\
15
\end{matrix}\right] 
$$

$$
\mathbf{W}^{\{2\}} = \left[\begin{matrix} 
10\\
10
\end{matrix}\right] 
$$

$$
\mathbf{b}^{\{2\}} = \left[\begin{matrix} 
-15\\
\end{matrix}\right] 
$$

In [9]:
# salida de XOR
y_xor = np.array([0., 1., 1., 0.])

# pesos y sesgo para la primera capa
# compuertas OR y NAND
W1 = np.array([[10, 10], [-10, -10]]).T
b1 = np.array([-5, 15])
# pesos y sesgo para la segunda capa
# compuerta AND
W2 = np.array([[10], [10]])
b2 = np.array([-15])

print('Pesos y sesgos para compuertas OR y NAND')
print('W_1 = [{0}{1}], b_1 = {2}'.format(W1[0, :], W1[1, :], b1))
print('Pesos y sesgos para compuerta AND')
print('W_2 = [{0}{1}], b_2 = {2}'.format(W2[0], W2[1], b2))
print('-----------------------------')
print('x_1 \tx_2 \ty\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    x1, x2 = x = X[i]
    y_hat = multi_layer(x, W1, b1, W2, b2)
    print(f'{x1}\t{x2}\t{y_xor[i]}\t{y_hat[0]}')

Pesos y sesgos para compuertas OR y NAND
W_1 = [[ 10 -10][ 10 -10]], b_1 = [-5 15]
Pesos y sesgos para compuerta AND
W_2 = [[10][10]], b_2 = [-15]
-----------------------------
x_1 	x_2 	y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	0.0	0.0
